## Clasificación de género con imágenes de manos — Modelo de Fusión

## Este notebook combina dos modelos previamente entrenados con diferentes preprocesamientos (GaussianBlur y Sobel) para mejorar la clasificación de género mediante una arquitectura de doble entrada.

## 1. Importación de librerías necesarias

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import GlobalAveragePooling2D
import matplotlib.pyplot as plt
print("Tensorflow version:", tf.__version__)
print("OpenCV version:", cv2.__version__)
print("Numpy version:", np.__version__)

## 2. Configuración de parámetros

In [ ]:
IMG_SIZE   = (224, 224)
BATCH_SIZE = 32
NUM_EPOCHS = 5

CSV_PATH   = "../../dataset/HandInfo.csv"
IMAGE_ROOT = "../../dataset/image"

## 3. Carga del CSV y codificación de etiquetas

In [ ]:
df = pd.read_csv(CSV_PATH)
df["label"] = df["gender"].map({"male": 0, "female": 1})
NUM_CLASSES = df["label"].nunique()

## 4. Función de normalización Z-score

In [ ]:
def z_score_normalization(image: np.ndarray) -> np.ndarray:
    mean = image.mean()
    std  = image.std()
    return (image - mean) / (std + 1e-7)

## 5. Funciones de preprocesamiento: GaussianBlur y Sobel

In [ ]:
def preprocess_gaussian(path_bytes: bytes) -> np.ndarray:
    path = path_bytes.decode("utf-8")
    img  = cv2.imread(path)
    if img is None:
        return np.zeros((*IMG_SIZE, 3), dtype=np.float32)
    img = cv2.resize(img, IMG_SIZE).astype(np.float32)
    img = cv2.GaussianBlur(img, (15, 15), 0)
    return z_score_normalization(img)

def preprocess_sobel(path_bytes: bytes) -> np.ndarray:
    path = path_bytes.decode("utf-8")
    img  = cv2.imread(path)
    if img is None:
        return np.zeros((*IMG_SIZE, 3), dtype=np.float32)
    img = cv2.resize(img, IMG_SIZE)
    gx  = cv2.Sobel(img, cv2.CV_32F, 1, 0, ksize=3)
    gy  = cv2.Sobel(img, cv2.CV_32F, 0, 1, ksize=3)
    img = np.sqrt(gx**2 + gy**2)
    return z_score_normalization(img)

# 6. Creación de datasets para ambas ramas

In [ ]:
def make_tf_preprocess(fn):
    def _fn(path, label):
        img = tf.numpy_function(fn, [path], tf.float32)
        img.set_shape((*IMG_SIZE, 3))
        return img, label
    return _fn

def create_ds(split, fn):
    folder = os.path.join(IMAGE_ROOT, split)
    files  = set(os.listdir(folder))
    subdf  = df[df["imageName"].isin(files)].sort_values("imageName")
    paths  = [os.path.join(folder,n) for n in subdf["imageName"]]
    labels = subdf["label"].values
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(make_tf_preprocess(fn), tf.data.AUTOTUNE)
    if split == "train":
        ds = ds.shuffle(1000)
    return ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

g_tr = create_ds("train", preprocess_gaussian)
g_va = create_ds("val",   preprocess_gaussian)
g_te = create_ds("test",  preprocess_gaussian)
s_tr = create_ds("train", preprocess_sobel)
s_va = create_ds("val",   preprocess_sobel)
s_te = create_ds("test",  preprocess_sobel)

## 7. Fusionar datasets de ambas ramas con `tf.data.Dataset.zip`

In [ ]:
train_ds = tf.data.Dataset.zip((g_tr, s_tr)) \
    .map(lambda g, s: ((g[0], s[0]), g[1]), tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.zip((g_va, s_va)) \
    .map(lambda g, s: ((g[0], s[0]), g[1]), tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.zip((g_te, s_te)) \
    .map(lambda g, s: ((g[0], s[0]), g[1]), tf.data.AUTOTUNE)
print("Train dataset size:", len(train_ds))
print("Validation dataset size:", len(val_ds))
print("Test dataset size:", len(test_ds))

## 8. Carga de modelos preentrenados y extracción de embeddings

In [ ]:
gauss_model = load_model("../Suavizadas/gaussian_model.h5")
sobel_model = load_model("../Detalladas/sobel_model.h5")

gauss_model.trainable = False
sobel_model.trainable = False

gap_g = next(l for l in gauss_model.layers if isinstance(l, GlobalAveragePooling2D))
gap_s = next(l for l in sobel_model.layers if isinstance(l, GlobalAveragePooling2D))

feat_g = Model(gauss_model.input, gap_g.output)
feat_s = Model(sobel_model.input, gap_s.output)

## 9. Definición del modelo de fusión con dos ramas de entrada

In [ ]:
inp_g = layers.Input((*IMG_SIZE, 3), name="in_gauss")
inp_s = layers.Input((*IMG_SIZE, 3), name="in_sobel")

emb_g = feat_g(inp_g)
emb_s = feat_s(inp_s)

x = layers.Concatenate()([emb_g, emb_s])
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
out = layers.Dense(NUM_CLASSES, activation="softmax")(x)

fusion = Model([inp_g, inp_s], out)

## 10. Compilación del modelo

In [ ]:
opt = SGD(learning_rate=0.01, momentum=0.9)
fusion.compile(
    optimizer=opt,
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

## 11. Callbacks para reducción de LR y evaluación en test

In [ ]:
lr_plateau = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.8,
    patience=2,
    verbose=1
)

class TestCallback(tf.keras.callbacks.Callback):
    def __init__(self, test_ds):
        super().__init__()
        self.test_ds = test_ds
        self.test_losses = []
        self.test_accuracies = []
    def on_epoch_end(self, epoch, logs=None):
        loss, acc = self.model.evaluate(self.test_ds, verbose=0)
        self.test_losses.append(loss)
        self.test_accuracies.append(acc)
        print(f"  → Test loss: {loss:.4f} — Test acc: {acc:.4f}")

test_cb = TestCallback(test_ds)

## 12. Entrenamiento del modelo

In [ ]:
history = fusion.fit(
    train_ds,
    validation_data=val_ds,
    epochs=NUM_EPOCHS,
    callbacks=[lr_plateau, test_cb],
)

## 13. Guardar modelo fusionado

In [ ]:
fusion.save("fusion_gauss_sobel.h5")

## 14. Visualización de resultados de entrenamiento

In [ ]:
epochs    = range(1, NUM_EPOCHS + 1)
train_acc = history.history["accuracy"]
test_acc  = test_cb.test_accuracies
train_loss= history.history["loss"]
test_loss = test_cb.test_losses

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Accuracy
ax1.plot(epochs, train_acc, marker='o', color='blue', label='Train')
ax1.plot(epochs, test_acc,  marker='o', color='red', label='Test')
ax1.set_title("Accuracy over epochs")
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Accuracy")
ax1.set_xticks(epochs)
ax1.legend()
ax1.grid(True)

# Loss
ax2.plot(epochs, train_loss, marker='o', color='blue', label='Train')
ax2.plot(epochs, test_loss,  marker='o', color='red', label='Test')
ax2.set_title("Loss over epochs")
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Loss")
ax2.set_xticks(epochs)
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()